In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
!pip install --upgrade pip
!pip install loguru opencv-python-headless tabulate netron
!pip install nvidia-pyindex
!pip install nvidia-tensorrt
!pip install torch-tensorrt

In [ ]:
!pip install accelerate
!pip install diffusers transformers tabulate

In [ ]:
!apt-get update && apt-get install -y graphviz
!pip install expecttest pydot

In [ ]:
import os; print(os.getenv("CUDA_MODULE_LOADING", None))
print(os.getenv("HF_HOME"))
print(os.getenv("TRANSFORMERS_CACHE"))
print(os.getenv("TORCH_HOME"))

In [ ]:
import torch
import torch.fx
import torch.nn as nn
print(torch.__version__)

In [ ]:
import tensorrt
print(tensorrt.__version__)

In [ ]:
import torch_tensorrt
print(torch_tensorrt.__version__)

In [ ]:
from torch_tensorrt.fx.tracer.acc_tracer import acc_normalizer, acc_ops, acc_shape_prop, acc_utils  # noqa: F401
from torch.fx.experimental.normalize import NormalizeArgs

import torch_tensorrt
from torch_tensorrt.fx.utils import LowerPrecision
import torch_tensorrt.fx.tracer.acc_tracer.acc_tracer as acc_tracer
from torch_tensorrt.fx import InputTensorSpec, TRTInterpreter, TRTModule
from torch_tensorrt.fx.tools.trt_splitter import TRTSplitter, TRTSplitterSetting

In [ ]:
from torch.fx.node import Argument, Target
from torch.fx import symbolic_trace, replace_pattern

from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union
import tensorrt as trt
from torch_tensorrt.fx.converters.converter_utils import SourceIR, get_trt_tensor, get_trt_plugin
from torch_tensorrt.fx.converter_registry import tensorrt_converter
from torch_tensorrt.fx.tracer.acc_tracer.acc_op_properties import AccOpProperty, register_acc_op_properties
from torch_tensorrt.fx.tracer.acc_tracer.acc_normalizer import (
    register_acc_op,
    register_acc_op_mapping,
    register_custom_acc_mapper_fn,
)
from torch_tensorrt.fx.types import (
    TRTNetwork,
    TRTTensor,
)
from torch_tensorrt.fx.converters.impl import activation

In [ ]:
from torch_tensorrt.fx.tracer.acc_tracer import acc_ops
from torch_tensorrt.fx.converter_registry import CONVERTERS
from torch_tensorrt.fx.tracer.acc_tracer.acc_normalizer import _acc_ops, _normalization_dict
import torch_tensorrt.fx.converter_registry as registry

In [ ]:
import tensorrt as trt
TRT_LOGGER = trt.Logger(trt.Logger.ERROR)
trt.init_libnvinfer_plugins(TRT_LOGGER, "")
print(f"Register libnvinfer plugins")
registry = trt.get_plugin_registry()
print(f"Registry: {registry}")
for plugin in registry.plugin_creator_list:
    print(plugin.name)


In [ ]:
registry.CONVERTERS.pop(acc_ops.expand)
for k in registry.CONVERTERS.keys():
    print(k)

In [ ]:
for op in list(CONVERTERS.keys()):
    if op == acc_ops.gelu:
        CONVERTERS.pop(op)
        print(f"removed converter {op}")

for op in list(_acc_ops):
    if op.__name__ == "gelu":
        _acc_ops.remove(op)
        print(f"removed acc_op: {op}")
        
for (op, target) in list(_normalization_dict.keys()):
    if "gelu" in str(target) or "GELU" in str(target):
        _normalization_dict.pop((op, target))
        print(f"removed normalization_dict op: {op}")

In [ ]:
import numpy as np

@register_acc_op_mapping(
    op_and_target=("call_function", torch.nn.functional.group_norm),
    arg_replacement_tuples=[
        ("input", "input"),
        ("num_groups", "num_groups"),
        ("weight", "weight"),
        ("bias", "bias"),
        ("eps", "eps"),
    ],
)
@register_acc_op
def group_norm(*, input, num_groups, weight=None, bias=None, eps=1e-05):
    return torch.nn.functional.group_norm(input, num_groups, weight=weight, bias=bias, eps=eps)


@tensorrt_converter(group_norm)
def acc_ops_group_norm(network, target, args, kwargs, name):
    input_val = kwargs["input"]
    weight = kwargs["weight"]
    bias = kwargs["bias"]
    shape = (input_val.shape[1],)
    if weight is None:
        weight = torch.ones(tuple([*input_val.shape])).to(
            torch_dtype_from_trt(input_val.dtype)
        )
    weight = get_trt_tensor(network, weight, f"{name}_weight")

    if bias is None:
        bias = torch.zeros(tuple([*input_val.shape])).to(
            torch_dtype_from_trt(input_val.dtype)
        )
    bias = get_trt_tensor(network, bias, f"{name}_bias")

    if not isinstance(input_val, trt.tensorrt.ITensor):
        raise RuntimeError(
            f"GroupNorm received input {input_val} that is not part "
            "of the TensorRT region!"
        )

    num_groups_field = trt.PluginField(
        "num_groups", np.array([kwargs["num_groups"]], dtype=np.int32), trt.PluginFieldType.INT32
    )
    eps_field = trt.PluginField(
        "eps", np.array([kwargs["eps"]], dtype=np.float32), trt.PluginFieldType.FLOAT32
    )

    field_collection = trt.PluginFieldCollection(
        [eps_field, num_groups_field]
    )

    try:
        plugin = get_trt_plugin("GroupNormalizationPlugin", field_collection, "1", "")
    except AssertionError:
        _LOGGER.error(
            "Unable to find group norm plugin, fall back to TensorRT implementation."
        )
        raise RuntimeError(
            f"Failed to build group norm plugin."
        )
    layer = network.add_plugin_v2([input_val, weight, bias], plugin)
    layer.name = name
    return layer.get_output(0)

In [ ]:
import requests
from PIL import Image

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
from diffusers import DiffusionPipeline
from diffusers import DDIMScheduler
from diffusers.pipelines.stable_diffusion import StableDiffusionPipeline

# Use the DDIMScheduler scheduler here instead
scheduler = DDIMScheduler.from_pretrained("stabilityai/stable-diffusion-2-1",
                                            subfolder="scheduler")

# pipe = DiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16)

# pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
# pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16, revision="fp16")

# pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16 , scheduler=scheduler)

# TensorRT
# pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1",
#                                                 custom_pipeline="stable_diffusion_tensorrt_txt2img",
#                                                 revision='fp16',
#                                                 torch_dtype=torch.float16,
#                                                 scheduler=scheduler,)
# pipe.set_cached_folder("stabilityai/stable-diffusion-2-1", revision='fp16',)

# CUDA
pipe = pipe.to("cuda")

# Custom attention
# pipe.unet.set_default_attn_processor()
# pipe.enable_attention_slicing()

In [ ]:

input = torch.randint(0, 1, (1, 14), dtype=torch.int32).to("cuda")
input2 = torch.randint(0, 1, (1, 14), dtype=torch.int32).to("cuda")

compile_spec = {
    "inputs": [
        torchtrt.Input(
            input.shape,
            dtype=input.dtype,
            format=torch.contiguous_format,
        ),
        torchtrt.Input(
            input.shape,
            dtype=input.dtype,
            format=torch.contiguous_format,
        ),
    ],
    "device": torchtrt.Device("cuda:0"),
    "enabled_precisions": {torch.float},
    "truncate_long_and_double": True,
    "ir": ir,
}
trt_mod = torchtrt.compile(model, **compile_spec)

model_outputs = model(input, input2)
trt_model_outputs = trt_mod(input, input2)
for key in model_outputs.keys():
    out, trt_out = model_outputs[key], trt_model_outputs[key]
    cos_sim = cosine_similarity(out, trt_out)
    assertions.assertTrue(
        cos_sim > COSINE_THRESHOLD,
        msg=f"HF BERT base-uncased TRT outputs don't match with the original model. Cosine sim score: {cos_sim} Threshold: {COSINE_THRESHOLD}",
    )

# Clean up model env
torch._dynamo.reset()

with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
import torch._dynamo
torch._dynamo.config.verbose = True
torch._dynamo.config.suppress_errors = True

pipe.unet.to(memory_format=torch.channels_last)
pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)
# model = pipe.unet._orig_mod

In [ ]:
pipe.unet = torch_tensorrt.dynamo.compile(
    pipe.unet,
    inputs,
    enabled_precisions={torch.half},
    debug=True,
    workspace_size=20 << 30,
    min_block_size=3,
    torch_executed_ops={},
)

In [ ]:
from PIL import Image
import torchvision.transforms.functional as F

# CLIP
# inputs = [torch.randn((1, 3, 224, 224), dtype=torch.float32, device='cuda')]

# UNET
# https://github.com/stochasticai/x-stable-diffusion/blob/main/TensorRT/convert_unet_to_tensorrt.py
# https://github.com/huggingface/diffusers/blob/666743302ff5bd1e02c204b81a80e566648d60de/examples/community/stable_diffusion_tensorrt_txt2img.py#L493
# latents_shape = (
#     args.batch_size * 2,
#     4,
#     args.img_size[0] // 8,
#     args.img_size[1] // 8,
# )
# embed_shape = (args.batch_size * 2, args.max_seq_length, 768)
# timestep_shape = (args.batch_size,)

# inputs = [
#     torch.randn(2, 4, 64, 64, dtype=torch.half, device='cuda:0'), 
#     torch.tensor([1, 3], dtype=torch.int32, device='cuda:0'), 
#     torch.randn(2, 77, 768, dtype=torch.half, device='cuda:0')
# ]


# TEXT ENCODER
# inputs = (torch.randn(2, 4, 64, 64, dtype=torch.half, device='cuda'), 
#           torch.randn(1, dtype=torch.half, device='cuda'), 
#           torch.randn(2, 77, 768, dtype=torch.half, device='cuda'))
# inputs

# fp16 (2.1.0.dev20230605+cu118)
# 4090: eager (sdv1-5): 30.6 it/s
# 4090: eager (sdv1-5): 30.6 it/s (w/ torch.compile, mode="reduce-overhead", fullgraph=True)

# fp32 (2.1.0.dev20230605+cu118)
# 4090: eager (sdv2-1): 5.6 it/s (w/ 2.0.1)
# 4090: eager (sdv2-1): 4.34 it/s (w/ attention-slicing)
# 4090: eager (sdv2-1): 7.2 it/s

# fp16 (2.1.0.dev20230605+cu118)
# 4090: eager (sdv2-1): 21.2 it/s
# 4090: eager (sdv2-1): 24.5 it/s (w/ torch.compile, mode="reduce-overhead", fullgraph=True)

# >> Inference
# with torch.inference_mode():
images = pipe(prompt=["fox jumped over dog"], num_inference_steps=200, num_images_per_prompt=1).images

# output.last_hidden_state.shape (1, 50, 768)
# output.pooler_output.shape  (1, 768)
# output.hidden_states = None
# output.attentions = None
# output

In [ ]:
images = pipe(prompt=["fox jumped over dog"], num_inference_steps=200, num_images_per_prompt=1).images

In [ ]:
images[0]

In [ ]:
# dir(pipe.unet)
emodel, _ = torch._dynamo.export(pipe, {"prompt": "test prompt"})
emodel

In [ ]:
torch._dynamo.explain(model, *inputs)

In [ ]:
# import onnxscript
# from onnxscript.onnx_opset import opset14 as op

# opset_version = 14
# custom_opset = onnxscript.values.Opset(domain="onnx-script", version=1)
# default_opset = onnxscript.values.Opset(domain="onnx", version=1)

# @onnxscript.script(custom_opset, default_opset)
# def ScaledDotProductAttention(Q, K, V, attn_mask=None, dropoout_p=0.0, is_causal=False):
#     # Efficient implementation equivalent to the following:
#     query_shape = list(Q.size())
#     key_shape = list(K.size())
#     L = query_shape[len(query_shape)-2]
#     S = key_shape[len(key_shape) - 2]
#     attn_mask = torch.ones(L, S, dtype=torch.bool).tril(diagonal=0) if is_causal else attn_mask
#     attn_mask = attn_mask.masked_fill(not attn_mask, -float('inf')) if attn_mask.dtype==torch.bool else attn_mask
#     attn_weight = torch.softmax((Q @ K.transpose(-2, -1) / math.sqrt(Q.size(-1))) + attn_mask, dim=-1)
#     attn_weight = torch.dropout(attn_weight, dropout_p)
#     return attn_weight @ V    

# def custom_scaled_dot_product_attention(g: jit_utils.GraphContext, Q, K, V):
#     return g.onnxscript_op(ScaledDotProductAttention, Q, K, V).setType(V.type())

# torch.onnx.register_custom_op_symbolic(
#     symbolic_name="aten::scaled_dot_product_attention",
#     symbolic_fn=custom_scaled_dot_product_attention,
#     opset_version=opset_version,
# )

In [ ]:
TRT_LOGGER = trt.Logger(trt.Logger.ERROR)
trt.init_libnvinfer_plugins(TRT_LOGGER, "")
print(f"Register libnvinfer plugins")
registry = trt.get_plugin_registry()
print(f"Registry: {registry}")

In [ ]:
for item in registry.plugin_creator_list:
    print(item.name, item.plugin_version, item.plugin_namespace)

In [ ]:
from torch.fx.node import Argument, Target
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union
from torch_tensorrt.fx.converters.converter_utils import SourceIR
from torch_tensorrt.fx.converter_registry import tensorrt_converter
from torch_tensorrt.fx.tracer.acc_tracer.acc_op_properties import AccOpProperty, register_acc_op_properties
from torch_tensorrt.fx.tracer.acc_tracer.acc_normalizer import (
    register_acc_op,
    register_acc_op_mapping,
    register_custom_acc_mapper_fn,
)
from torch_tensorrt.fx.types import (
    TRTNetwork,
    TRTTensor,
)

In [ ]:
import logging
from torch_tensorrt.dynamo.backend._settings import CompilationSettings
from torch_tensorrt.dynamo.backend.utils import prepare_inputs, prepare_device
from torch_tensorrt.dynamo.backend.backends import aot_torch_tensorrt_aten_backend
from torch_tensorrt.dynamo.backend._defaults import (
    PRECISION,
    DEBUG,
    MAX_WORKSPACE_SIZE,
    MIN_BLOCK_SIZE,
    PASS_THROUGH_BUILD_FAILURES,
)
from torch._dynamo.backends.common import fake_tensor_unsupported
from torch._functorch.aot_autograd import aot_module_simplified, make_boxed_compiler

from nos.compilers.trt.backends import nos_tensorrt_backend

logger = logging.getLogger(__name__)


# # @td.register_backend(name="torch_tensorrt")
# @fake_tensor_unsupported
# def nos_tensorrt_backend(
#     gm: torch.fx.GraphModule, sample_inputs: Sequence[torch.Tensor], **kwargs
# ):
#     DEFAULT_BACKEND = aot_torch_tensorrt_aten_backend
#     print(gm)

#     return DEFAULT_BACKEND(gm, sample_inputs, **kwargs)


def create_backend(
    precision: LowerPrecision = PRECISION,
    debug: bool = DEBUG,
    workspace_size: int = MAX_WORKSPACE_SIZE,
    min_block_size: int = MIN_BLOCK_SIZE,
    torch_executed_ops: Sequence[str] = set(),
    pass_through_build_failures: bool = PASS_THROUGH_BUILD_FAILURES,
    **kwargs,
):
    """Create torch.compile backend given specified arguments

    Args:
        precision: Model Layer precision
        debug: Whether to print out verbose debugging information
        workspace_size: Workspace TRT is allowed to use for the module (0 is default)
        min_block_size: Minimum number of operators per TRT-Engine Block
        torch_executed_ops: Sequence of operations to run in Torch, regardless of converter coverage
        pass_through_build_failures: Whether to fail on TRT engine build errors (True) or not (False)
    Returns:
        Backend for torch.compile
    """
    if debug:
        logger.setLevel(logging.DEBUG)

    settings = CompilationSettings(
        debug=debug,
        precision=precision,
        workspace_size=workspace_size,
        min_block_size=min_block_size,
        torch_executed_ops=torch_executed_ops,
        pass_through_build_failures=pass_through_build_failures,
    )

    return partial(
        nos_tensorrt_backend,
        settings=settings,
    )

In [ ]:
import torch
import logging
import collections.abc
import torch_tensorrt
from functools import partial

from typing import Any, Sequence
from torch_tensorrt import EngineCapability, Device
from torch_tensorrt.fx.utils import LowerPrecision


torch._dynamo.config.log_level = logging.INFO
torch._dynamo.config.verbose = True
torch._dynamo.config.suppress_errors = True

debug = True
custom_backend = create_backend(
    precision=LowerPrecision.FP16,
    debug=debug,
    workspace_size=MAX_WORKSPACE_SIZE,
    min_block_size=MIN_BLOCK_SIZE,
    torch_executed_ops=[],
)


# torch._dynamo.eval_frame.remove_from_cache(pipe.unet)
with torch.inference_mode():
    pipe.unet = torch.compile(pipe.unet, backend=custom_backend)

    # Ensure compilation occurs by calling the function with provided inputs
    pipe.unet(*args.values())

In [ ]:
%%timeit -n 10
with torch.inference_mode():
    pipe.unet(*args.values())

In [ ]:

# def gelu_fn(x):
#     """
#     https://github.com/geohot/tinygrad/blob/18892242b006785d4e92abae7c792e7874c17df9/tinygrad/tensor.py#L522
#     """
#     return 0.5 * x * (1 + (x * 0.7978845608 * (1 + 0.044715 * x * x)).tanh())


# # py/torch_tensorrt/fx/converters/impl/activation.py
# def gelu(
#     network: TRTNetwork,
#     target: Target,
#     source_ir: Optional[SourceIR],
#     name: str,
#     input_val: TRTTensor,
# ):
#     operation_type = trt.ActivationType.CLIP

#     def gelu_dyn_range_fn(dyn_range):
#         return gelu_fn(dyn_range[0]), gelu_fn(dyn_range[1])

#     return convert_activation(
#         network,
#         target,
#         source_ir,
#         name,
#         operation_type,
#         input_val,
#         dyn_range_fn=gelu_dyn_range_fn,
#     )

# # py/torch_tensorrt/fx/converters/nn_ops_converters.py
# @tensorrt_converter(torch.nn.functional.gelu)
# @tensorrt_converter(torch.nn.modules.activation.GELU)
# def nn_ops_gelu(network, submod, args, kwargs, layer_name):
#     # args/kwargs should have already been normalized to kwargs
#     assert len(args) == 0

#     return gelu(
#         network=network,
#         target="torch.nn.modules.activation.GELU",
#         source_ir=SourceIR.NN,
#         name=layer_name,
#         input_val=kwargs["input"],
#     )


# # py/torch_tensorrt/fx/tracer/acc_tracer/acc_ops.py
# @register_acc_op_properties(AccOpProperty.pointwise, AccOpProperty.unary)
# @register_acc_op_mapping(op_and_target=("call_function", torch.nn.functional.gelu))
# @register_acc_op
# def acc_ops_gelu(*, input, approximate='none'):
#     assert approximate == 'none', "Only approximate='none' supported"
#     return gelu_fn(input)

In [ ]:
# @register_acc_op_mapping(
#     op_and_target=("call_function", torch.nn.functional.group_norm),
#     arg_replacement_tuples=[
#         ("input", "input"),
#         ("num_groups", "num_groups"),
#         ("weight", "weight"),
#         ("bias", "bias"),
#         ("eps", "eps"),
#     ],
# )
# @register_acc_op
# def group_norm(*, input, num_groups, weight=None, bias=None, eps=1e-05):
#     return torch.nn.functional.group_norm(
#         input, num_groups, weight=weight, bias=bias, eps=eps
#     )

In [ ]:
# import math

# @register_acc_op_mapping(
#     op_and_target=("call_function", torch._C._nn.scaled_dot_product_attention),
#     arg_replacement_tuples=[
#         ("query", "query"),
#         ("key", "key"),
#         ("value", "value"),
#         ("attn_mask", "attn_mask"),
#         ("dropout_p", "dropout_p"),
#         ("is_causal", "is_causal"),
#     ],
# )
# @register_acc_op
# def scaled_dot_product_attention(*, query, key, value, attn_mask=None, dropout_p=0.0, is_causal=False):
#     # Efficient implementation equivalent to the following:
#     query_shape = list(query.size())
#     key_shape = list(key.size())
#     L = query_shape[len(query_shape)-2]
#     S = key_shape[len(key_shape) - 2]
#     attn_weight = (query @ key.transpose(-2, -1) / math.sqrt(query.size(-1)))
#     if attn_mask is not None:
#         attn_mask = torch.ones(L, S, dtype=torch.bool).tril(diagonal=0) if is_causal else attn_mask
#         attn_mask = attn_mask.masked_fill(not attn_mask, -float('inf')) if attn_mask.dtype==torch.bool else attn_mask
#         attn_weight += attn_mask
#     attn_weight = torch.softmax(attn_weight, dim=-1)
#     attn_weight = torch.nn.functional.dropout(attn_weight, dropout_p, training=False)
#     return attn_weight @ value

In [ ]:
for k in registry.CONVERTERS.keys():
    print(k)

In [ ]:
pipe.dtype

In [ ]:
from torch import fx
from transformers.utils.fx import HFTracer

device = "cuda"
model = pipe.vae

# https://github.com/huggingface/diffusers/blob/666743302ff5bd1e02c204b81a80e566648d60de/examples/community/stable_diffusion_tensorrt_txt2img.py#L493
H, W = 512, 512
B = 1
sample = torch.rand(B, 3, H // 8, W // 8, dtype=torch.half, device=device)
args = {
    'sample': sample,
}

# Order matters
concrete_args = {
    "sample_posterior": False,
    "return_dict": True,
    "generator": None,
}
inputs = [
    args['sample'], 
 ]

tracer = HFTracer()
with torch.inference_mode():
    traced_graph = tracer.trace(model, concrete_args=concrete_args, dummy_inputs=args)
    traced_model = torch.fx.GraphModule(model, traced_graph)
# print(traced_graph.print_tabular())

traced_model.config = model.config
# The model class must be stored as an attribute to allow model deserialization, which uses trace, and thus
# _generate_dummy_input, where the model class is needed.
traced_model.class_for_deserialization = model.__class__
traced_model.device = model.device

In [ ]:
type(traced_model)

In [ ]:
model

In [ ]:
from typing import Any, Callable, Dict, List, Optional, Type, Union
import inspect

from transformers.models.auto import get_values
from transformers.models.auto.modeling_auto import MODEL_FOR_MULTIPLE_CHOICE_MAPPING_NAMES
from transformers.utils.fx import HFTracer, _generate_random_int, _gen_constructor_wrapper
from torch import fx
from torch.fx import Graph, GraphModule, Proxy, Tracer


class NOSTracer(HFTracer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def is_leaf_module(self, m: torch.nn.Module, module_qualified_name: str) -> bool:
        print(f"module: {m.__module__}")
        is_leaf = (super().is_leaf_module(m, module_qualified_name))
        # is_leaf = is_leaf and not m.__module__.startswith("diffusers.models.attention_processor")
        # if is_leaf:
        #     print(f"leaf module: {m.__module__}")
        return is_leaf

device = "cuda"
model = pipe.unet

# https://github.com/huggingface/diffusers/blob/666743302ff5bd1e02c204b81a80e566648d60de/examples/community/stable_diffusion_tensorrt_txt2img.py#L493
H, W = 512, 512
B = 1
D = 1024  # D = (v1-5: 768, v2-1: 1024)
unet_dim = 4 # pipe.unet.in_channels
text_maxlen = 77
# timestep = pipe.scheduler.timesteps[0]

sample = torch.rand(2 * B, unet_dim, H // 8, W // 8, dtype=torch.half, device=device)
timestep = torch.tensor([1, ], dtype=torch.int32, device=device)
encoder_hidden_states = torch.rand((2 * B, text_maxlen, D), dtype=torch.half, device=device)  # 2B, 77, 1024

# timestep_condition = torch.cat([noise_level_embed, text_pooler_out], dim=1)
args = {
    'sample': sample,
    'timestep' : timestep,
    'encoder_hidden_states': encoder_hidden_states,
}

# Order matters
concrete_args = {
    'class_labels': None,
    'timestep_cond' : None, # timestep_condition,
    'attention_mask': None,
    'cross_attention_kwargs': None,
    'added_cond_kwargs': None,
    'down_block_additional_residuals': None,
    'mid_block_additional_residual': None,
    'encoder_attention_mask':None,
    'return_dict': True,
    'hidden_states': None,
    'cond_proj_dim' : 1,
    'condition': None,
    'use_ada_layer_norm': True,
    'input_ndim': 4,
}
inputs = [
    args['sample'], 
    args['timestep'], 
    args['encoder_hidden_states']
]


# tracer = NOSTracer()
# with torch.inference_mode():
#     traced_graph = tracer.trace(model, concrete_args={**concrete_args}, dummy_inputs=args)
#     traced_model = torch.fx.GraphModule(model, traced_graph)
# # print(">>" * 80)
# # traced_graph.print_tabular()

# traced_model.config = model.config
# # The model class must be stored as an attribute to allow model deserialization, which uses trace, and thus
# # _generate_dummy_input, where the model class is needed.
# traced_model.class_for_deserialization = model.__class__
# traced_model.device = model.device

In [ ]:
# from transformers.models.clip.modeling_clip import CLIPVisionTransformer, CLIPAttention, CLIPVisionEmbeddings, CLIPEncoder
from typing import (
    Any,
    Callable,
    cast,
    Dict,
    Iterable,
    Optional,
    Sequence,
    Set,
    Tuple,
    Type,
    Union,
)
from torch_tensorrt.fx.tracer.acc_tracer import acc_normalizer, acc_ops, acc_shape_prop, acc_utils  # noqa: F401
from torch_tensorrt.fx.tracer.acc_tracer.acc_tracer import _remove_assertions, _remove_exceptions, _replace_tensor_meta_with_rank, _replace_transpose_last_dims, AccRewritingTracer
from torch.fx.experimental.normalize import NormalizeArgs



def my_compiler(gm: torch.fx.GraphModule, inputs):
    # acc_tracer is a custom fx tracer that maps nodes whose targets are PyTorch operators
    # to acc ops.
    # traced = acc_tracer.trace(model, concrete_args=input_kwargs)
    with torch.inference_mode():
        gm = acc_tracer.trace(
            gm, inputs,
        )

    return gm


import logging
torch._dynamo.eval_frame.remove_from_cache(pipe.unet)
torch._dynamo.config.log_level = logging.INFO
torch._dynamo.config.verbose = True
torch._dynamo.config.suppress_errors = True

pipe.unet.to(memory_format=torch.channels_last)
pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)
# model = torch.compile(pipe.unet, backend=my_compiler)
# model(*inputs)

In [ ]:
type(traced_model)

In [ ]:
pipe.unet

In [ ]:
import math

NORM = math.sqrt(512)
torch.fx.wrap('len')

def scaled_dot_product_attention(query, key, value, norm, attn_mask=None, dropout_p=0.0, is_causal=False):
    attn_weight = (query / norm).half() @ key.transpose(-2, -1)
    attn_weight = torch.nn.functional.softmax(attn_weight, dim=-1)
    attn_weight = torch.nn.functional.dropout(attn_weight, dropout_p, training=False)
    return attn_weight @ value

def nn_functional_sdpa(q, k, v):
    return torch.nn.functional.scaled_dot_product_attention(q, k, v)

def ours_sdpa(q, k, v, **kwargs):
    # B, C, K, E = q.shape
    # norm = math.sqrt(torch.tensor([E,]).half().cuda())
    return scaled_dot_product_attention(q, k, v, NORM)


def replace_patterns(traced_model, patterns, new_op):
    found = 0
    # Go through all the nodes in the Graph
    for idx, n in enumerate(traced_model.graph.nodes):
        # If the target matches one of the patterns
        if any(n.target == pattern for pattern in patterns):
            print("Found: ", n.target, n.name, n.op, n.args, n.kwargs)
            found += 1
            # Set the insert point, add the new node, and replace all uses
            # of `n` with the new node
            with traced_model.graph.inserting_after(n):
                new_node = traced_model.graph.call_function(new_op, n.args, n.kwargs)
                n.replace_all_uses_with(new_node)
            # Remove the old node from the graph
            traced_model.graph.erase_node(n)
    print(f"Found: {found}")
    _ = traced_model.recompile()
    
sdpa_patterns = set([
    torch._C._nn.scaled_dot_product_attention, 
    torch.nn.functional.scaled_dot_product_attention
])
replace_patterns(traced_model, sdpa_patterns, ours_sdpa)
# replace_pattern(traced_model, nn_functional_sdpa, ours_sdpa)
# print(traced_model)

In [ ]:
def gelu(x):
    return torch.nn.functional.gelu(x)

def gelu_fn(x):
    """
    https://github.com/geohot/tinygrad/blob/18892242b006785d4e92abae7c792e7874c17df9/tinygrad/tensor.py#L522
    """
    return 0.5 * x * (1 + (x * 0.7978845608 * (1 + 0.044715 * x * x)).tanh())

gelu_patterns = set([
    torch.nn.functional.gelu
])
replace_patterns(traced_model, gelu_patterns, gelu_fn)
for idx, n in enumerate(traced_model.graph.nodes):
    print(n.name)
#     if "gelu" in n.name:
#         print(n.op, n.target, n.name, n.args, n.kwargs)

In [ ]:
with torch.inference_mode():
    output = model(*inputs)
output

In [ ]:
with torch.inference_mode():
    output = traced_model()
output

In [ ]:
%%timeit -n 10
with torch.inference_mode():
    model(*inputs)

In [ ]:
%%timeit -n 10
with torch.inference_mode():
    traced_model()

In [ ]:
with torch.inference_mode():
    trt_compiled = torch_tensorrt.fx.compile(
        traced_model, 
        inputs,
        min_acc_module_size = 100, 
        max_batch_size = 2048, 
        max_workspace_size=33_554_432, 
        explicit_batch_dimension=False, 
        lower_precision=LowerPrecision.FP32, 
        verbose_log=False, 
        timing_cache_prefix='', 
        save_timing_cache=False, 
        cuda_graph_batch_size=- 1, 
        dynamic_batch=True, 
        is_aten=False, 
        use_experimental_fx_rt=False, 
        correctness_atol=0.1, 
        correctness_rtol=0.1
    )

In [ ]:
# from torch_tensorrt.fx import compile
# from torch_tensorrt.fx.tools.common_fx2trt import InputTensorSpec

# # fx_only = torch.fx.symbolic_trace(model)
# # fx_only(input)

# trt_mod = compile(pipe.unet, 
#                   [InputTensorSpec.from_tensor(input) for input in inputs], 
#                   # min_acc_module_size=1, 
#                   explicit_batch_dimension=True, 
#                   dynamic_batch=False)

In [ ]:
# from transformers.models.clip.modeling_clip import CLIPVisionTransformer, CLIPAttention, CLIPVisionEmbeddings, CLIPEncoder
from typing import (
    Any,
    Callable,
    cast,
    Dict,
    Iterable,
    Optional,
    Sequence,
    Set,
    Tuple,
    Type,
    Union,
)
from torch_tensorrt.fx.tracer.acc_tracer import acc_normalizer, acc_ops, acc_shape_prop, acc_utils  # noqa: F401
from torch_tensorrt.fx.tracer.acc_tracer.acc_tracer import _remove_assertions, _remove_exceptions, _replace_tensor_meta_with_rank, _replace_transpose_last_dims, AccRewritingTracer
from torch.fx.experimental.normalize import NormalizeArgs


# acc_tracer is a custom fx tracer that maps nodes whose targets are PyTorch operators
# to acc ops.
# traced = acc_tracer.trace(model, concrete_args=input_kwargs)
with torch.inference_mode():
    trt_traced = acc_tracer.trace(
        traced_model, inputs,
    )

In [ ]:
print(trt_traced.graph)

In [ ]:
%%timeit -n 10
model(*inputs)

In [ ]:
with torch.inference_mode():
    output = trt_traced(*inputs)
output

In [ ]:
%%timeit -n 10
trt_traced(*inputs)

In [ ]:
# Splitter will split the model into several submodules. The name of submodules will
# be either `run_on_acc_{}` or `run_on_gpu_{}`. Submodules named `run_on_acc_{}` can
# be fully lowered to TensorRT via fx2trt while submodules named `run_on_gpu_{}` has
# unsupported ops and can't be lowered by fx2trt. We can still run `run_on_gpu_{}`
# submodules on Gpu if ops there have cuda implementation, the naming is a bit
# confusing and we'll improve it.

settings = TRTSplitterSetting()
# settings.use_implicit_batch_dim = False
# settings.use_experimental_rt = False

with torch.inference_mode():
    splitter = TRTSplitter(trt_traced, inputs, settings=settings)
splitter

In [ ]:
# Preview functionality allows us to see what are the supported ops and unsupported
# ops. We can optionally the dot graph which will color supported ops and unsupported
# ops differently.
_ = splitter.node_support_preview(dump_graph=True)

In [ ]:
# !dot -Tpdf -Gdpi=1000 node_support.dot > node_support.pdf

In [ ]:
# from IPython.display import Image
# import pydot

# graphs = pydot.graph_from_dot_file("node_support.dot")
# Image(graphs[0].create_png())
# from IPython.display import SVG

# SVG("node_support.svg")

In [ ]:
print("Non Acc Nodes")
print(splitter.non_acc_submodule_name)

# splitter.sample_input
# splitter.split_preview()
# dir(splitter)
print("Acc Nodes")
print(len(splitter.acc_nodes))

In [ ]:
# splitter.remove_small_acc_subgraphs?
# dir(splitter)

In [ ]:
# Split.
split_mod = splitter()

In [ ]:
type(split_mod)

In [ ]:
# After split we have three submodules, _run_on_acc_0 and _run_on_gpu_1.
print(split_mod.graph)

In [ ]:
def get_submod_inputs(_mod, _submod, _inputs):
    acc_inputs = None

    def get_input(self, inputs):
        nonlocal acc_inputs
        acc_inputs = inputs

    handle = _submod.register_forward_pre_hook(get_input)
    with torch.inference_mode():
        _mod()
    handle.remove()
    return acc_inputs


# Since the model is splitted into three segments. We need to lower each TRT eligible segment.
# If we know the model can be fully lowered, we can skip the splitter part.
for name, _ in split_mod.named_children():
    print(f"Splitting {name}")
    if "_run_on_acc" in name:
        submod = getattr(split_mod, name)

        # Get submodule inputs for fx2trt
        acc_inputs = get_submod_inputs(split_mod, submod, inputs)

        # fx2trt replacement
        interp = TRTInterpreter(
            submod,
            InputTensorSpec.from_tensors(acc_inputs),
            explicit_batch_dimension=True,
        )
        r = interp.run(lower_precision=LowerPrecision.FP16)
        trt_mod = TRTModule(*r)
        setattr(split_mod, name, trt_mod)

In [ ]:
# from pathlib import Path
# from nos.constants import NOS_MODELS_DIR

# model_dir = Path(NOS_MODELS_DIR, f"cache/{MODEL_NAME}")
# model_dir.mkdir(parents=True, exist_ok=True)

# lowered_model_output = split_mod(*inputs)

# Save and load model
# W, H = 224, 224
# model_id = MODEL_NAME.replace("/", "-") + "_" + f"{W}x{H}" + "_" + "fp16"
# filename = f"{model_dir}/{model_id}.torchtrt.pt"
filename = "vae_224x224.torchtrt.pt"
torch.save(split_mod, filename)

In [ ]:
print(f"Loading torchtrt filename: {filename}")
reload_trt_mod = torch.load(filename)

In [ ]:
with torch.inference_mode():
    reload_model_output = reload_trt_mod(*inputs)
reload_model_output

In [ ]:
# Make sure the results match
with torch.inference_mode():
    regular_model_output = model(*inputs)
regular_model_output

In [ ]:
%%timeit -n 100
with torch.inference_mode():
    _ = model(*inputs)

In [ ]:
%%timeit -n 100
with torch.inference_mode():
    _ = reload_trt_mod(*inputs)

In [ ]:
for o1, o2 in zip(reload_model_output, regular_model_output):
    torch.testing.assert_close(
        o1.cpu().float(), o2.cpu().float(), rtol=2e-02, atol=2e-02, equal_nan=True
    )

In [ ]:
%%timeit -n 100
with torch.inference_mode():
    outputs = model(*inputs)

In [ ]:
# Monkeypatch backbone
model.backbone = reload_trt_mod

In [ ]:
%%timeit -n 100
inputs_fp16 = [inp.half() for inp in inputs]
with torch.inference_mode():
    outputs = model(*inputs_fp16)
outputs

In [ ]:
from transformers.utils.fx import symbolic_trace


# traced_model = symbolic_trace(model, input_names=["input_ids", "attention_mask", "position_ids", "output_attentions", "output_hidden_states", "return_dictd"])
# traced = fx.symbolic_trace(
#     model, 
#     concrete_args={'sample': torch.zeros(2, 4, 64, 64, dtype=torch.half, device='cuda:0'), 
#                    'timestep': torch.tensor([1, 3], dtype=torch.int32, device='cuda:0'),
#                    'encoder_hidden_states': torch.randn(2, 77, 768, dtype=torch.half, device='cuda:0'),
#                    'timestep_cond': None, 
#                    'attention_mask': None,
#                    'cross_attention_kwargs': None,
#                    'added_cond_kwargs': None,
#                    'down_block_additional_residuals': None,
#                    'mid_block_additional_residual': None,
#                    'encoder_attention_mask': None}
# )
# print(traced.code)

In [ ]:
# scripted_model = torch.jit.script(model.backbone)
# compile_settings = {
#     "inputs": [torch_tensorrt.Input(
#         min_shape=[1, 3, 480, 640],
#         opt_shape=[1, 3, 960, 1280],
#         max_shape=[4, 3, 960, 1280],
#     )],
#     "enabled_precisions": {torch.float}  # Run with FP16 (torch.half)
# }
# trt_ts_module = torch_tensorrt.compile(scripted_model, compile_settings)
# input_data = torch.randn((1, 3, 480, 640))
# input_data = input_data.to("cuda")
# pytorch_out = model.forward(input_data)
# torch_tensorrt_out = trt_ts_module(input_data)


# print('PyTorch output: \n', pytorch_out[0, :, :, 0])

# trt_ts_module = torch_tensorrt.compile(model.backbone,
#     # If the inputs to the module are plain Tensors, specify them via the `inputs` argument:
#     inputs = [ # Provide example tensor for input shape or...
#         torch_tensorrt.Input( # Specify input object with shape and dtype
#             min_shape=[1, 3, 480, 640],
#             opt_shape=[1, 3, 480, 640],
#             max_shape=[1, 3, 960, 1280],
#             # For static size shape=[1, 3, 224, 224]
#             dtype=torch.float) # Datatype of input tensor. Allowed options torch.(float|half|int8|int32|bool)
#     ],

#     # For inputs containing tuples or lists of tensors, use the `input_signature` argument:
#     # Below, we have an input consisting of a Tuple of two Tensors (Tuple[Tensor, Tensor])
#     # input_signature = ( (torch_tensorrt.Input(shape=[1, 3, 224, 224], dtype=torch.half),
#     #                      torch_tensorrt.Input(shape=[1, 3, 224, 224], dtype=torch.half)), ),

#     enabled_precisions = {torch.float}, # Run with FP16
# )